In [1]:
import warnings
warnings.filterwarnings('ignore')

import copy
import datetime
import h5py
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import sys

from cv2 import resize
from datetime import datetime
from gc import collect
from os import cpu_count
from sklearn.model_selection import train_test_split
from time import sleep
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary
from tqdm import tqdm

sys.path.append(f"{os.getcwd()}/working ViT/")
sys.path.append(f"{os.getcwd()}/mltfm/")
from VisionTransformer_working import VisionTransformer as vit_old

2023-02-01 10:23:22.291866: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-01 10:23:22.388700: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-01 10:23:22.388719: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-01 10:23:22.801403: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

ModuleNotFoundError: No module named 'VisionTransformer_working'

In [ ]:
random_seed = 1
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
cudnn.benchmark = True

In [ ]:
collect()
torch.cuda.empty_cache()
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device('cpu')
print(f"Running on device: {device}")

## Idea

Optimization for a linear combination of similar tasks often yields models that are more accurate, generalize better and require less training data. A possible multi-task objective for TFM could be
<h3><center>$\mathcal{L} = \alpha_{1}$$\cdot MSE + \alpha_{2}$$\cdot DTMA + \alpha_{3}$$\cdot DDA$</center></h3>
where
<h3><center>$\boldsymbol\alpha > 0, \space\space\sum_{i=1}^{3} \alpha_{i} = 1$</center></h3>


Try model trained on multi-task objective.

In [ ]:
vit_multi = vit_old(dspl_size=104,
                  patch_size=8,
                  embed_dim=128,
                  depth=12,
                  n_heads=8,
                  mlp_ratio=4.,
                  p=0.,
                  attn_p=0.,
                  drop_path=0.).float()

In [ ]:
path_to_multi_pth = '/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM/working ViT/logs_and_weights/ViT-2023-Jan-09 11:57:14/ViT-2023-Jan-09 11:57:14_best_val_loss_0.003215498429.pth'

In [ ]:
if torch.cuda.is_available():
    vit_multi.load_state_dict(torch.load(path_to_multi_pth)['best_model_weights'], strict=True)
else:
    vit_multi.load_state_dict(torch.load(path_to_multi_pth, map_location=torch.device('cpu'))['best_model_weights'], strict=True)        

In [ ]:
dspl = np.array(h5py.File(f'data/extended data/allDisplacements.h5')['dspl'])
trac_separated = np.array(h5py.File(f'data/extended data/allTractions.h5')['trac'])

dspl = np.moveaxis(np.concatenate([dspl[i] for i in range(dspl.shape[0])], axis=0), 3, 1)
trac_separated = np.moveaxis(np.concatenate([trac_separated[i] for i in range(trac_separated.shape[0])], axis=0), 3, 1)

dspl_train = dspl[0:100, :, :, :]
dspl_val = dspl[100:110, :, :, :]
trac_train = trac_separated[0:100, :, :, :]
trac_val = trac_separated[100:110, :, :, :]

X_train = torch.from_numpy(dspl_train).float()
Y_train = torch.from_numpy(trac_train).float()
X_val = torch.from_numpy(dspl_val).float()
Y_val = torch.from_numpy(trac_val).float()

In [ ]:
from MultiTask import multi_task_loss, dtma, dda, append_predictions_and_targets

In [ ]:
vit_multi.eval()
loss = multi_task_loss
pred_vit_multi = vit_multi(X_train)

In [ ]:
pred_vit_multi.shape

In [ ]:
%matplotlib inline

fig, axs = plt.subplots(1, 2, figsize=(9, 5))
fig.tight_layout(pad=2, w_pad=2, h_pad=2)

axs[0].quiver(pred_vit_multi[0, 0, :, :].detach().numpy(), pred_vit_multi[0, 1, :, :].detach().numpy(), scale=3)
axs[0].set_title('appended_predictions', {'fontsize': 11})

axs[1].quiver(Y_train[0, 0, :, :], Y_train[0, 1, :, :], scale=3)
axs[1].set_title('appended_targets', {'fontsize': 11})

In [ ]:
fig, axs = plt.subplots(50, 2, figsize=(10, 120))
fig.tight_layout(pad=2, w_pad=2, h_pad=2)

for i in range(50):
    axs[i, 0].quiver(appended_predictions[0, i, 0, :, :].detach().numpy(), appended_predictions[0, i, 1, :, :].detach().numpy(), scale=1)
    axs[i, 0].set_title('appended_predictions', {'fontsize': 11})

    axs[i, 1].quiver(appended_targets[0, i, 0, :, :], appended_targets[0, i, 1, :, :], scale=1)
    axs[i, 1].set_title('appended_targets', {'fontsize': 11})


In [ ]:
get_ipython().run_line_magic('matplotlib', 'notebook')

# Plots
fig, axs = plt.subplots(1,2, figsize=(8, 8))
fig.tight_layout(pad=3, w_pad=3, h_pad=3)

axs[0].quiver(X_test[0, 0, :, :], X_test[0, 1, :, :], scale=1)
axs[0].set_title('PointForcemeshes[0]', {'fontsize': 11})

trac_test = torch.sum(trac_separated_test, dim=1)

axs[1].quiver(X_test[10, 0, :, :], X_test[10, 1, :, :], scale=1)
axs[1].set_title('PointForcemeshes[1]', {'fontsize': 11})

In [ ]:
vit_weights = vit_old(dspl_size=104,
                      patch_size=8,
                      embed_dim=128,
                      depth=12,
                      n_heads=8,
                      mlp_ratio=4.,
                      p=0.,
                      attn_p=0.,
                      drop_path=0.).double()

In [ ]:
path_to_pth = 'working ViT/logs_and_weights/ViT-2022-Dec-10 23:18:41/ViT-2022-Dec-10 23:18:41_best_val_loss_2.365828e-06.pth'

In [ ]:
if torch.cuda.is_available():
    vit_weights.load_state_dict(torch.load(path_to_pth)['best_model_weights'], strict=False)
else:
    vit_weights.load_state_dict(torch.load(path_to_pth, map_location=torch.device('cpu'))['best_model_weights'], strict=False)        

In [ ]:
os.getcwd()

In [ ]:
dspl = h5py.File('data/extended data/displacements_1.h5')["data"]
trac_separated = h5py.File('data/extended data/sep_tractions_1.h5')["data"]

dspl_ = np.moveaxis(np.array(dspl[0:50, :, :, :]),3 ,1)
trac_separated_ = np.moveaxis(np.array(trac_separated[0:50, :, :, :]), 4, 2)

X_test = torch.from_numpy(dspl_).double().to(device)
trac_separated_test = torch.from_numpy(trac_separated_).double().to(device)

In [ ]:
print(X_test.shape)
print(trac_separated_test.shape)

In [ ]:
vit_weights.eval()
pred_vit = vit_weights(X_test[:, :, :, :])
# pred_cnn = cnn_weights.predict(dspl[0:50, :, :, :])

In [ ]:
get_ipython().run_line_magic('matplotlib', 'notebook')

# Plots
fig, axs = plt.subplots(3,2, figsize=(8, 10))
fig.tight_layout(pad=3, w_pad=3, h_pad=3)

axs[0, 0].quiver(pleaase[0, :, :, 0], pleaase[0, :, :, 1], scale=5)
axs[0, 0].set_title('trac_separated_test[0, 0]', {'fontsize': 11})

plt.imshow(pleaase[0, :, :, 2], cmap='hot', interpolation='none')

axs[0, 1].plt.imshow(pleaase[0, :, :, 2], cmap='hot', interpolation='nearest')
axs[0, 1].set_title('appended_predictions[0, 0]', {'fontsize': 11})

axs[1, 0].quiver(trac_separated_test[0, 10, 0, :, :].detach().numpy(), trac_separated_test[0, 10, 1, :, :].detach().numpy(), scale=10)
axs[1, 0].set_title('trac_separated_test[0, 13, 0]', {'fontsize': 11})

axs[1, 1].quiver(trac_separated_test[0, 15, 0, :, :].detach().numpy(), trac_separated_test[0, 15, 1, :, :].detach().numpy(), scale=10)
axs[1, 1].set_title('trac_separated_test[0, 20, 0]', {'fontsize': 11})

axs[2, 0].quiver(trac_separated_test[0, 20, 0, :, :].detach().numpy(), trac_separated_test[0, 20, 0, :, :].detach().numpy(), scale=23)
axs[2, 0].set_title('Y_test[0, 49, 0]', {'fontsize': 11})

axs[2, 1].quiver(pred_vit[0,0,:,:].detach().numpy(), pred_vit[0,1,:,:].detach().numpy(), scale=13)
axs[2, 1].arrow(43, 18, Y_test[0, 0, 43, 18], 1,fc='red')
axs[2, 1].set_title('pred_vit', {'fontsize': 11})

In [ ]:
tryout = h5py.File('/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM/data/extended data/try_displacements_1.h5')

In [ ]:
tryout['data']

In [ ]:
dtma_ex = dtma(appended_predictions, trac_separated_test)

In [ ]:
dtma_ex

In [ ]:
trac_separated_test[:,:].shape

In [ ]:
dtma_ex

In [ ]:
pred_vit.shape

In [ ]:
trac_separated_test.shape

In [ ]:
get_ipython().run_line_magic('matplotlib', 'notebook')

# Plots
fig, axs = plt.subplots(1, 2, figsize=(5, 5))
fig.tight_layout(pad=3, w_pad=3, h_pad=3)

axs[0].quiver(pred_vit[0, 0, 35, 12].detach().numpy(), pred_vit[0, 1, 35, 12].detach().numpy(), scale=0.05)
axs[0].set_title('pred_vit[0, 0, 0]', {'fontsize': 11})

axs[1].quiver(Y_test[0, 0, 35, 12].detach().numpy(), Y_test[0, 0, 35, 12].detach().numpy(), scale=0.05)
axs[1].set_title('trac_separated_test[0, 0, 0]', {'fontsize': 11})

In [ ]:
os.getcwd()

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
event_acc = EventAccumulator('/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM/working ViT/logs_and_weights/ViT-2023-Jan-09 11:57:14')
event_acc.Reload()
# Show all tags in the log file
print(event_acc.Tags())

In [ ]:
event_acc.Scalars(tag='train_loss')

In [ ]:
writer = SummaryWriter(log_dir=f'/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM/working ViT/logs_and_weights/ViT-2023-Jan-11 10:36:58')

In [ ]:
writer['train_loss']